In [1]:
%pip install torchmetrics 



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 14.8 MB/s 


In [2]:
import tqdm 
import matplotlib.pyplot as plt
import torchvision 
import torchmetrics
import torch 
from torch import nn
from torch.nn import functional as F


In [3]:
train_data = torchvision.datasets.KMNIST('./data', train=True, download=True)

  0%|          | 0/18165135 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/train-images-idx3-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/29497 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/train-labels-idx1-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/3041136 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/5120 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/KMNIST/raw



<h2> Iterative dataset vs map-style </h2>

Iterative = base form, can be access by index ---> return a iterator

In [8]:
for img, lab in train_data: break
# just for extract in funzy way first img & label --> they are now torch.tensor
# img = pil.image.image, but a pil_img is just a tensor 

8

<h1>Normalization</h1>

any time that we download the dataset we can pass a sequence of transformation, AKA just a set of funct that normalize, rescale and change type of our img.
We can add data augmentation also in this part, some transformation are created to pytorch tensor and other are desing to PIL img.

Exist both function version and class version. 
The transformation can be compose, aka mixed togheter 

MODEL VS DATA AUG = can be mixed, see next cell


In [4]:
from torchvision import transforms as T

In [5]:
#instanciate the obj
#apply to the element
function = T.ToTensor()  #  convert img form pil to tensor
function(img).shape      #  apply to img
                         #  using first index as channel in torch, half problematic
                         #  this slow down the gpu power --->search other info
# alternative way
# T.ToTensor()(img).shape 

NameError: ignored


For data augmentation: don't do it for test part, or at least not any kind of transofrmation, so define two set of funct

In [13]:


train_transforms = T.Compose([
    T.RandomRotation(15), # specify larger amount of rotation to apply ---> + or - 15 degree
    T.ToTensor()
])
# application directly to dataset: 
train_data = torchvision.datasets.KMNIST('./data',
                                          train=True, 
                                          transform=train_transforms)

In [14]:
# train_data = iterator
# train_loader = iterator
# difference: loader iters over minibatch, data over sample.
#             Loader also shuffle data each time that finish to iterate
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
x_batch, y_batch = next(iter(train_loader))
x_batch.shape
# [64 img, 1 channel, 28x28 pixel in each img]

torch.Size([64, 1, 28, 28])

In [15]:
test_data = torchvision.datasets.KMNIST('./data', train=False, transform=T.ToTensor())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

<h1>Net, building and training</h1>

In [16]:
conv2d = nn.Conv2d(1, 16, 5)
print(conv2d(x_batch).shape )

# notice that we've lost some pixel ---> no padding
conv2d_padding = nn.Conv2d(1, 16, 5, padding=2)
print(conv2d_padding(x_batch).shape)

# increase step of kernel : reduce dim: stride
conv2d_padding_and_stride = nn.Conv2d(1, 16, 5, padding=2, stride=2)
print(conv2d_padding_and_stride(x_batch).shape)

# parameters in layers: 
for name, par in conv2d_padding_and_stride.named_parameters():
  print(f"name: {name}, shape {par.shape}")

torch.Size([64, 16, 24, 24])
torch.Size([64, 16, 28, 28])
torch.Size([64, 16, 14, 14])
name: weight, shape torch.Size([16, 1, 5, 5])
name: bias, shape torch.Size([16])


In [41]:
# OTHER LAYERS

dropout_layer = nn.Dropout(0.4)
x = torch.randn((3, 2))
print(dropout_layer(x))           # --> notice 0 on some value, randomly
# removing dropout for testing
dropout_layer.eval()        # --> put in testing mode the entire layer, just like removing
dropout_layer(x)
#notice the difference of value: that because the dropout scale value during train



tensor([[-0.6692, -0.0000],
        [-0.0000, -2.1857],
        [ 0.0000, -0.0000]])


tensor([[-0.4015, -1.6652],
        [-0.4808, -1.3114],
        [ 0.2698, -0.6307]])

In [46]:
# focus on trainable and non-trainable parameters
batch_norm_layer = nn.BatchNorm2d(16) 
# init alpha and beta at 0 and 1
# 16 = output of previous layer 
batch_norm_layer(conv2d_padding_and_stride(x_batch)).mean([0, 2, 3])

for p  in batch_norm_layer.parameters():
  print(p, p.shape)
# vs

for p in batch_norm_layer.buffers(): # parameters that are not trainable
# not optimizated during train 
  print(p)
  print(p.shape)
# neither conv and drop has buffers
# 

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True) torch.Size([16])
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True) torch.Size([16])
tensor([ 0.0084,  0.0163, -0.0080,  0.0072,  0.0158,  0.0289,  0.0226, -0.0007,
        -0.0012, -0.0170, -0.0252,  0.0102, -0.0308,  0.0093,  0.0028,  0.0065])
torch.Size([16])
tensor([0.9016, 0.9021, 0.9015, 0.9025, 0.9054, 0.9072, 0.9054, 0.9025, 0.9030,
        0.9043, 0.9041, 0.9028, 0.9071, 0.9028, 0.9007, 0.9027])
torch.Size([16])
tensor(1)
torch.Size([])


In [7]:
# CONTAINERS 
# module, sequential, 
# ModuleList --> list of module, in general it's use for several module with different parameters configuaration  
# they work exactly like a layer

class SimpleCnn(nn.Module): 
  def __init__(self):
    super().__init__() 
    
    self.conv1 = nn.Conv2d(1, 16, 5, padding=2)
    self.bn1 = nn.BatchNorm2d(16)
    
    self.conv2  =nn.Conv2d(16, 32, 5, padding=2)
    self.bn2 = nn.BatchNorm2d(32)
    
    self.max_pool = nn.MaxPool2d(2)  # take the max of each kernel 2x2 --> dimezza
    
    self.fc1 = nn.Linear(32*7*7, 100)
    self.fc2 = nn.Linear(100, 10)
    self.drop = nn.Dropout(0.3)

  def forward(self, x):
    
    x = F.relu(self.bn1(self.conv1(x)))
    x = self.max_pool(x) 

    x = F.relu(self.bn2(self.conv2(x)))
    x = self.max_pool(x)
    # Output shape: (batch_size, output channel from conv2, 7*7) 
    # 7x7 --> 28/2 by first max_pool, and /2 again for 2 max_pool
    x = x.reshape((-1, 32*7*7))
    # opt 2: x = x.mean([2, 3]) across h, w
    x = self.drop(F.relu(self.fc1(x)))
    return self.fc2(x) 
    # don't need to return a relu/softmax.
    # for computational reason, it's better to return always the logits


In [8]:
cnn = SimpleCnn()

len(list(cnn.parameters())) # --> 12 tensor

12

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cnn = SimpleCnn().to(device)

In [20]:
loss = nn.CrossEntropyLoss()
opt = torch.optim.Adam(cnn.parameters()) # iterate through the parameters, faster that parallelaize 

def accuracy(net, loader):
  acc = torchmetrics.Accuracy().to(device)
  for x_b, y_b in loader:
    x_b, y_b = x_b.to(device), y_b.to(device)
    y_pred = cnn(x_b)
    _ = acc(y_pred, y_b)
  return acc.compute()

In [19]:
accuracy(cnn, test_loader)

tensor(0.9283, device='cuda:0')

In [21]:
for epoch in range(5): 
  
  cnn.train()
  for x_b, y_b in tqdm.tqdm(train_loader):


    x_b = x_b.to(device)
    y_b = y_b.to(device)

    opt.zero_grad() # look over all the parameters and zero the grad
    y_pred = cnn(x_b)
    l = loss(y_pred, y_b)
    l.backward()
    opt.step()
  
  cnn.eval()
  print(f'accuracy at epoch {epoch}: {accuracy(cnn, test_loader)}')

100%|██████████| 938/938 [00:12<00:00, 75.74it/s]


accuracy at epoch 0: 0.9506999850273132


100%|██████████| 938/938 [00:12<00:00, 76.72it/s]


accuracy at epoch 1: 0.9513999819755554


100%|██████████| 938/938 [00:12<00:00, 77.62it/s]


accuracy at epoch 2: 0.9542999863624573


100%|██████████| 938/938 [00:12<00:00, 77.17it/s]


accuracy at epoch 3: 0.9545000195503235


100%|██████████| 938/938 [00:12<00:00, 77.15it/s]


accuracy at epoch 4: 0.9564999938011169
